# Import

In [1]:
import pandas as pd
import numpy as np
import requests 
import json
import ijson
import duckdb
from rapidfuzz import process, fuzz
import matplotlib.pyplot as plt

# LittleSis

## Functions

In [2]:
# Function to fetch entities by a list of IDs
def get_entities_LS(entity_ids_list):
    json_file = "entities.json"
    entity_ids_list = set(entity_ids_list)  # Convert to set for fast lookup
    

    with open(json_file, "r", encoding="utf-8") as file:
        json_data = json.load(file)

        if isinstance(json_data, list):
            filtered_data= pd.DataFrame()

            for entity in json_data:
                if entity.get("attributes").get("id") in entity_ids_list:
                    attributes = entity.get("attributes")
                    entity_info = {
                            "ID": attributes.get("id"),
                            "Name": attributes.get("name"),
                            "Aliases":';'.join( attributes.get("aliases")),
                            "Type": attributes.get("primary_ext"),
                            "Types_other":'; '.join(attributes.get("types")),
                            "Description": attributes.get("blurb"),
                            "Updated": attributes.get("updated_at"),
                            "Links": entity.get("links").get("self"),
                            "Source":"Littlesis"
                        }   
                    new_row = pd.DataFrame([entity_info])
                    filtered_data = pd.concat([filtered_data,new_row], ignore_index=True).drop_duplicates(subset="ID", keep="last")
        else:
            print("Invalid JSON format: Expected a list of entities.")
            return filtered_data # Return empty DataFrame if invalid format
        
    # Return results in Pandas DataFrame
    return filtered_data



In [3]:
# Merge 2 Dataframes - adding unique rows only
def add_to_df(source, extension):
    return pd.concat([source,extension], ignore_index=True).drop_duplicates(subset="ID", keep="last")

In [4]:
def get_relationships_LS(entity_ids, related_entities=None, rel_categories=None, json_file="relationships.json"):

    entity_ids = set(entity_ids)
    if related_entities is None:
        related_entities = set()  # Use a set for fast lookups

    related_entities.update(entity_ids if isinstance(entity_ids, set) else {entity_ids}) 

    # Mapping relationship category IDs to descriptions
    rel_category_mapping = {
        1: "Position", 2: "Education", 3: "Membership", 4: "Family", 5: "Donation/Grant",
        6: "Service/Transaction", 7: "Lobbying", 8: "Social", 9: "Professional", 10: "Ownership",
        11: "Hierarchy", 12: "Generic"
    }

    filtered_relationships = []  # Store results efficiently

    with open(json_file, "r", encoding="utf-8") as file:
        objects = ijson.items(file, "item")  # Streaming JSON parser
        
        for rel in objects:
            attributes = rel.get("attributes", {})
            category_id = attributes.get("category_id")
            category_attributes = attributes.get("category_attributes")
            
            # Check category filter (if provided)
            if rel_categories and category_id not in rel_categories:
                continue

            entity1_id, entity2_id = attributes.get("entity1_id"), attributes.get("entity2_id")

            # Ensure we are selecting only relationships for the target entity
            if entity1_id not in entity_ids and entity2_id not in entity_ids:
               continue

            # Identify the related entity (not the source entity)
            related_entity_id = entity2_id if entity1_id in entity_ids else entity1_id
            related_entities.add(related_entity_id) 

            # Store relationship data in a dictionary
            filtered_relationships.append({
                "ID": rel.get("id"),
                "Entity1_ID": entity1_id,
                "Entity2_ID": entity2_id,
                "Category_ID": category_id,
                "Category_desc": rel_category_mapping.get(category_id, "Unknown"),
                "Description": attributes.get("description"),
                "Description_1": attributes.get("description1"),
                "Description_2": attributes.get("description2"),
                "is_current": attributes.get("is_current"),
                "is_board": category_attributes.get("is_board", "")if isinstance(category_attributes, dict) else "",
                "is_executive": category_attributes.get("is_executive", "")if isinstance(category_attributes, dict) else "",
                "is_employee": category_attributes.get("is_employee", "")if isinstance(category_attributes, dict) else "",
                "amount": attributes.get("amount"),
                "currency": attributes.get("currency"),
                "Start_date": attributes.get("start_date"),
                "End_date": attributes.get("end_date"),
                "Updated": attributes.get("updated_at"),
                "Link": rel.get("self"),
            })

    # Convert the list of relationships into a DataFrame efficiently
    return pd.DataFrame(filtered_relationships)

In [5]:
"""# Getting relationship + related IDs
def get_relationships(entity_id,entity_id_list):
    url = f"https://littlesis.org/api/entities/{entity_id}/relationships"  # API URL for entity data
    response = requests.get(url)
    json_data = response.json()

    if entity_id_list is None:
        entity_id_list = []  # Initialize a new list if no argument is passed
        
    #page count
    currentPage=1
    lastPage = json_data.get("meta").get("pageCount")

    relationship_id_list = []
    entity_id_list.append(entity_id) #create list of IDs + add source ID
    filtered_data = pd.DataFrame()


    list_item = 0
    rel_category_mapping = {
        1: "Position",
        2: "Education",
        3: "Membership",
        4: "Family",
        5: "Donation/Grant",
        6: "Service/Transaction",
        7: "Lobbying",
        8: "Social",
        9: "Professional",
        10: "Ownership",
        11: "Hierarchy",
        12: "Generic"
    }
 
    
    while currentPage <= lastPage:  # go through all pages
        url = f"https://littlesis.org/api/entities/{entity_id}/relationships?page={currentPage}"
        response = requests.get(url)
        json_data = response.json()

        ######################       
        currentPage += 1
        list_item += 1
        ######################

        for rel in json_data.get("data"): #iterating relationships on 1 page
            if rel.get("id") not in relationship_id_list:
                relationship_id_list.append(rel.get("id"))

                entity1_id = rel.get("attributes").get("entity1_id") 
                entity2_id = rel.get("attributes").get("entity2_id") 
                category_attributes = rel.get("attributes").get("category_attributes")

                #add only related id (not soruce)
                if entity1_id == entity_id: 
                    entity_id_list.append(entity2_id)
                else:
                    entity_id_list.append(entity1_id)

                
                relationship_info = {
                    "ID":rel.get("id"),
                    "Entity1_ID": rel.get("attributes").get("entity1_id"),
                    "Entity2_ID": rel.get("attributes").get("entity2_id"),
                    "Category_ID": rel.get("attributes").get("category_id") ,
                    "Category_desc": rel_category_mapping.get(rel.get("attributes").get("category_id")) ,
                    "Description": rel.get("attributes").get("description"),
                    "Description_1": rel.get("attributes").get("description1"),
                    "Description_2": rel.get("attributes").get("description2"),
                    "is_current": rel.get("attributes").get("is_current"),
                    "is_board": rel.get("attributes").get("category_attributes").get("is_board") if isinstance(category_attributes, dict) else "",
                    "is_executive": rel.get("attributes").get("category_attributes").get("is_executive") if isinstance(category_attributes, dict) else "",
                    "is_employee":rel.get("attributes").get("category_attributes").get("is_employee") if isinstance(category_attributes, dict) else "",
                    "amount":rel.get("attributes").get("amount"),
                    "currency":rel.get("attributes").get("currency"),
                    "Start_date":rel.get("attributes").get("start_date"),
                    "End_date": rel.get("attributes").get("end_date"),
                    "Updated": rel.get("attributes").get("updated_at"),
                    "Link": rel.get("self") ,
                }

                new_row = pd.DataFrame([relationship_info])
                filtered_data = pd.concat([filtered_data,new_row], ignore_index=True).drop_duplicates(subset="ID", keep="last")
            else: continue
    entity_id_list[:] = list(set(entity_id_list))# unique related IDs

    # Return results in Pandas DataFrame
    return filtered_data
"""

'# Getting relationship + related IDs\ndef get_relationships(entity_id,entity_id_list):\n    url = f"https://littlesis.org/api/entities/{entity_id}/relationships"  # API URL for entity data\n    response = requests.get(url)\n    json_data = response.json()\n\n    if entity_id_list is None:\n        entity_id_list = []  # Initialize a new list if no argument is passed\n        \n    #page count\n    currentPage=1\n    lastPage = json_data.get("meta").get("pageCount")\n\n    relationship_id_list = []\n    entity_id_list.append(entity_id) #create list of IDs + add source ID\n    filtered_data = pd.DataFrame()\n\n\n    list_item = 0\n    rel_category_mapping = {\n        1: "Position",\n        2: "Education",\n        3: "Membership",\n        4: "Family",\n        5: "Donation/Grant",\n        6: "Service/Transaction",\n        7: "Lobbying",\n        8: "Social",\n        9: "Professional",\n        10: "Ownership",\n        11: "Hierarchy",\n        12: "Generic"\n    }\n \n    \n    w

## Execution

In [6]:
#DataFrame init
df_entities = pd.DataFrame()
df_relationships = pd.DataFrame()


In [7]:
#get relationships (level 1)
trump_related_ids= set()
df_relationships = add_to_df(df_relationships,get_relationships_LS([15108],trump_related_ids))

In [8]:
print(df_relationships.shape[0],df_entities.shape[0])

2799 0


In [9]:
#get related entities (level 1)
df_entities = add_to_df(df_entities,get_entities_LS(trump_related_ids))

In [10]:
# get Trump connected level 2 relationships (positions + ownership only)
trump_lvl2_related_ids = set()
df_relationships = add_to_df(df_relationships,get_relationships_LS(trump_related_ids,trump_lvl2_related_ids,rel_categories={1,10,11}))

C:\Users\rober\AppData\Local\Temp\ipykernel_19328\1457603684.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([source,extension], ignore_index=True).drop_duplicates(subset="ID", keep="last")


In [11]:
# get level 2 related entities
df_entities = add_to_df(df_entities,get_entities_LS(trump_lvl2_related_ids))

In [12]:
print(df_relationships.shape[0],df_entities.shape[0])

13520 10743


| #  | Name                  | Description |
|----|-----------------------|-------------|
| 1  | Position (P+O/P+P only) | When a person has a place in an organizational hierarchy, usually carrying a title. Positions can be paid or unpaid. **Examples:** CEO, Director, Trustee, Chief Counsel, Professor, etc. |
| 2  | Education (P+O only)   | When a person attends a school or educational program as a student. |
| 3  | Membership            | When a person is a member of a membership organization but doesn’t hold a position, or when an organization is a member of a larger coalition or association. **Examples:** AFL-CIO, NRA, National Association of Manufacturers, etc. |
| 4  | Family (P+P only)     | When two people are part of the same family. **Examples:** children, spouses, cousins, siblings, etc. |
| 5  | Donation/Grant        | A gift transfer of money, goods, or services with nothing due in return. **Examples:** political funding, contributions to charities, government grants, prizes. |
| 6  | Service/Transaction   | An exchange of money, goods, or services of about equal value. **Examples:** purchases, consulting, contract work, accounting, trades, etc. |
| 7  | Lobbying              | When an organization directly lobbies a government agency or official. **Examples:** organization that employs lobbyists in-house, lobbying firm hired by an organization. |
| 8  | Social (P+P only)     | When two people are socially acquainted. **Examples:** friends, rivals, lovers, tennis partners, etc. |
| 9  | Professional (P+P only) | When two people have a direct working or business relationship. **Examples:** co-writers, business partners, mentors, etc. |
| 10 | Ownership (O+O/P+O only) | When a person or organization has full or partial ownership of an organization. **Examples:** sole proprietor, limited partners, shareholders, etc. |
| 11 | Hierarchy (O+O only)  | When an organization is the parent or child of another organization. **Examples:** foundation, lobbying arm, political arm, etc. |
| 12 | Generic               | A generic type for affiliations that do not fit into any of the regular types of relationships. |


pro organizace: 11, 10
pro lidi: 1, 5,6, 7, 9

### Export raw

In [13]:
df_relationships.to_csv('relationships.csv',index=False)
df_entities.to_csv('entities.csv',index=False)

## Transformace

In [14]:
entities = pd.read_csv('entities.csv')
relationships = pd.read_csv('relationships.csv')

In [15]:
# description concatenation
relationships["Description_1"] = relationships.apply(
    lambda row: "; ".join([val for val in [row["Description_1"], row["Description_2"]] if pd.notna(val)])
    if row["Description_1"] != row["Description_2"] else row["Description_1"],
    axis=1
)

In [16]:
#pridani sloupce id_all (spojeni obou ID pro snazsi vyhledavani)
relationships["ID_all"] = relationships["Entity1_ID"].astype(str) + ', ' + relationships["Entity2_ID"].astype(str)

In [17]:
# oprava minusovych hodnot v amount
relationships["amount"] = relationships["amount"].abs()

In [18]:
entities

,ID,Name,Aliases,Type,Types_other,Description,Updated,Links,Source
0,1,Walmart Inc.,"IRS EIN 71-0415188;Wal-Mart;Wal-Mart Stores, I...",Org,Organization; Business; Public Company,Retail merchandising,2024-07-15T21:29:07Z,https://littlesis.org/entities/1-Walmart_Inc.,Littlesis
1,2,ExxonMobil,Esso;Exxon Corporation;Exxon Mobil;Exxon Mobil...,Org,Organization; Business; Public Company,"Oil and gas exploration, production, and marke...",2025-02-21T23:06:40Z,https://littlesis.org/entities/2-ExxonMobil,Littlesis
2,3,Chevron,Chevron;Chevron Corporation;ChevronTexaco;IRS ...,Org,Organization; Business; Public Company,Energy Company,2025-02-21T23:06:54Z,https://littlesis.org/entities/3-Chevron,Littlesis
3,4,General Motors Company,GM;General Motors;General Motors Company;Gener...,Org,Organization; Business; Public Company,automobile manufacturer,2025-02-03T15:40:42Z,https://littlesis.org/entities/4-General_Motor...,Littlesis
4,5,ConocoPhillips,Conoco Inc.;ConocoPhillips;ConocoPhillips Hold...,Org,Organization; Business; Public Company,Texas-based oil and gas corporation,2023-09-02T18:25:19Z,https://littlesis.org/entities/5-ConocoPhillips,Littlesis
...,...,...,...,...,...,...,...,...,...
10738,455281,PREVENTING INFORMATION REMOVAL AND NABBING HAR...,PREVENTING INFORMATION REMOVAL AND NABBING HAR...,Org,Organization,NaN,2025-02-24T17:44:38Z,https://littlesis.org/entities/455281-PREVENTI...,Littlesis
10739,455282,Strategies for Investigating and Eliciting Inf...,Strategies for Investigating and Eliciting Inf...,Org,Organization,DARPA program,2025-02-25T18:01:27Z,https://littlesis.org/entities/455282-Strategi...,Littlesis
10740,455317,Amy Gleason,Amy Gleason,Person,Person; Business Person,"Nashville tech executive, DOGE administrator",2025-02-26T02:43:24Z,https://littlesis.org/entities/455317-Amy_Gleason,Littlesis
10741,455346,Tesla Automation GmbH,Grohmann Engineering;Tesla Automation GmbH,Org,Organization; Business,Tesla German subsidiary,2025-02-27T22:23:50Z,https://littlesis.org/entities/455346-Tesla_Au...,Littlesis


In [19]:
relationships

,ID,Entity1_ID,Entity2_ID,Category_ID,Category_desc,Description,Description_1,Description_2,is_current,is_board,is_executive,is_employee,amount,currency,Start_date,End_date,Updated,Link,ID_all
0,232530,50049,15108,9,Professional,Tevfik Arif and Donald Trump are/were a bus...,a business associate,a business associate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-03-08T21:43:35Z,https://littlesis.org/relationships/232530,"50049, 15108"
1,232531,49948,15108,9,Professional,Felix Sater and Donald Trump are/were a bus...,a business associate,a business associate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-08T01:57:29Z,https://littlesis.org/relationships/232531,"49948, 15108"
2,297889,15108,29581,5,Donation/Grant,Donald Trump gave money to Solutions America...,Campaign Contribution,Campaign Contribution,NaN,NaN,NaN,NaN,6500.0,usd,1999-00-00,2004-00-00,2011-01-06T21:31:22Z,https://littlesis.org/relationships/297889,"15108, 29581"
3,297890,15108,28801,5,Donation/Grant,Donald Trump gave money to Rudy Giuliani,Campaign Contribution,Campaign Contribution,NaN,NaN,NaN,NaN,1000.0,usd,1999-00-00,2007-05-15,2017-04-24T14:05:26Z,https://littlesis.org/relationships/297890,"15108, 28801"
4,297891,15108,13422,5,Donation/Grant,Donald Trump gave money to Frank A Lobiondo,Campaign Contribution,Campaign Contribution,NaN,NaN,NaN,NaN,8250.0,usd,1996-00-00,2007-00-00,2011-01-06T21:33:07Z,https://littlesis.org/relationships/297891,"15108, 13422"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13515,2001527,139907,14664,1,Position,Kelly Loeffler has a position (administrator)...,administrator,NaN,True,NaN,True,NaN,NaN,NaN,2025-02-20,NaN,2025-03-07T16:53:27Z,https://littlesis.org/relationships/2001527,"139907, 14664"
13516,2001593,157922,38938,1,Position,Laura Ingraham has/had a position (Position) ...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-08T02:12:42Z,https://littlesis.org/relationships/2001593,"157922, 38938"
13517,2001594,49998,38938,1,Position,Maria Bartiromo has/had a position (Position)...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-08T02:13:20Z,https://littlesis.org/relationships/2001594,"49998, 38938"
13518,2001651,455536,38804,1,Position,Riccardo Biasini has/had a position (software...,software engineer,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,2025-03-09T17:40:42Z,https://littlesis.org/relationships/2001651,"455536, 38804"


### Export transformed

In [20]:
relationships.columns = relationships.columns.str.lower()
entities.columns = entities.columns.str.lower()

relationships.to_csv('relationships.csv',index=False)
entities.to_csv('entities.csv',index=False)

### SQL Analyza

In [21]:
duckdb.query("SELECT category_desc, count(*) as count FROM relationships group by all having count(*) >1 order by 2 desc").to_df()

,category_desc,count
0,Position,9865
1,Donation/Grant,2196
2,Ownership,809
3,Hierarchy,221
4,Generic,140
5,Professional,129
6,Service/Transaction,69
7,Social,63
8,Family,22
9,Membership,4


In [22]:
orgs_rels = duckdb.query(
    "SELECT distinct r.* FROM entities e inner JOIN relationships r "
    "   on e.ID = r.entity1_ID or e.ID = r.entity2_ID "
    "WHERE type = 'Org' "
    ).to_df()

orgs_rels

,id,entity1_id,entity2_id,category_id,category_desc,description,description_1,description_2,is_current,is_board,is_executive,is_employee,amount,currency,start_date,end_date,updated,link,id_all
0,48,1045,2,1,Position,Rex Tillerson had a position (Director) at E...,Director,Director,False,True,False,<NA>,NaN,None,2004-00-00,None,2018-02-15T02:58:35Z,https://littlesis.org/relationships/48,"1045, 2"
1,30421,15557,2,1,Position,G O Wilhelm has/had a position (Lobbyist) at ...,Lobbyist,Lobbyist,<NA>,<NA>,<NA>,True,NaN,None,None,None,2008-11-10T22:45:06Z,https://littlesis.org/relationships/30421,"15557, 2"
2,30423,15559,2,1,Position,J O Luby Jr has/had a position (Lobbyist) at ...,Lobbyist,Lobbyist,<NA>,<NA>,<NA>,True,NaN,None,None,None,2008-11-10T22:45:06Z,https://littlesis.org/relationships/30423,"15559, 2"
3,30445,15567,2,1,Position,J A Cole has/had a position (Lobbyist) at Ex...,Lobbyist,Lobbyist,<NA>,<NA>,<NA>,True,NaN,None,None,None,2008-11-10T22:45:13Z,https://littlesis.org/relationships/30445,"15567, 2"
4,30451,15571,2,1,Position,L D Jackson has/had a position (Lobbyist) at ...,Lobbyist,Lobbyist,<NA>,<NA>,<NA>,True,NaN,None,None,None,2008-11-10T22:45:18Z,https://littlesis.org/relationships/30451,"15571, 2"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9464,1989052,268377,402358,1,Position,Chris Krebs has/had a position (Position) at ...,,None,<NA>,<NA>,<NA>,<NA>,NaN,None,None,None,2024-11-18T17:44:53Z,https://littlesis.org/relationships/1989052,"268377, 402358"
9465,1989870,102193,451894,1,Position,Donald Trump Jr. has/had a position (Position...,,None,<NA>,<NA>,<NA>,<NA>,NaN,None,None,None,2024-11-28T02:51:15Z,https://littlesis.org/relationships/1989870,"102193, 451894"
9466,1999059,222770,455136,10,Ownership,Tom Homan is/was an owner of Homeland Strate...,,None,<NA>,<NA>,<NA>,<NA>,NaN,None,None,None,2025-02-18T17:26:58Z,https://littlesis.org/relationships/1999059,"222770, 455136"
9467,1999981,455281,455251,11,Hierarchy,PREVENTING INFORMATION REMOVAL AND NABBING HAR...,,None,<NA>,<NA>,<NA>,<NA>,NaN,None,None,None,2025-02-24T17:44:38Z,https://littlesis.org/relationships/1999981,"455281, 455251"


In [24]:
lvl1 = duckdb.query("SELECT* FROM relationships where id_all like '%15108%'").to_df()
pd.DataFrame(lvl1["category_desc"].value_counts())

,count
category_desc,
Donation/Grant,2196
Position,146
Generic,140
Professional,129
Service/Transaction,69
Social,63
Ownership,28
Family,22
Membership,4


In [25]:
duckdb.query(
    "SELECT  category_id ,category_desc, count(*) FROM orgs_rels group by 1,2 "
).to_df()

,category_id,category_desc,count_star()
0,1,Position,8274
1,12,Generic,59
2,10,Ownership,808
3,3,Membership,4
4,11,Hierarchy,221
5,2,Education,2
6,6,Service/Transaction,27
7,5,Donation/Grant,74


In [26]:

duckdb.query(
    "SELECT  category_id, Category_desc, count(*), FROM relationships r left join entities e on e.ID = r.entity1_ID"
    " left join entities e2 on e2.ID = r.entity2_ID "
    "WHERE Entity1_ID = 15108 or entity2_ID = 15108 "
    "group by category_id, Category_desc order by 3 "
).to_df()


,category_id,category_desc,count_star()
0,2,Education,2
1,3,Membership,4
2,4,Family,22
3,10,Ownership,28
4,8,Social,63
5,6,Service/Transaction,69
6,9,Professional,129
7,12,Generic,140
8,1,Position,146
9,5,Donation/Grant,2196


In [27]:
df_rel_l2 = duckdb.query(
    "SELECT * FROM relationships where id_all not like '%15108%' "
).to_df()

In [28]:
duckdb.query(
    "SELECT * FROM relationships WHERE is_current = 'True' and end_date is not null"
).to_df()

,id,entity1_id,entity2_id,category_id,category_desc,description,description_1,description_2,is_current,is_board,is_executive,is_employee,amount,currency,start_date,end_date,updated,link,id_all
0,1562917,138555,15108,6,Service/Transaction,John Dowd and Donald Trump did business,legal advisor; client,client,True,<NA>,<NA>,<NA>,NaN,None,2017-06-00,2018-03-00,2019-06-23T15:02:41Z,https://littlesis.org/relationships/1562917,"138555, 15108"
1,636,1356,27,1,Position,Edward M Liddy has a position (Director) at ...,Director,Director,True,True,False,False,NaN,None,2007-00-00,2008-10-27,2015-03-31T15:34:53Z,https://littlesis.org/relationships/636,"1356, 27"
2,36134,19016,19015,1,Position,H Rodgin Cohen has a position (Chairman) at ...,Chairman,Chairman,True,<NA>,<NA>,False,NaN,None,2000-00-00,2009-00-00,2011-02-21T14:05:40Z,https://littlesis.org/relationships/36134,"19016, 19015"
3,92853,34615,34616,1,Position,Wayne LaPierre has a position (Executive Vice...,Executive Vice President,Executive Vice President,True,<NA>,True,<NA>,NaN,None,None,2024-01-31,2024-01-30T01:44:32Z,https://littlesis.org/relationships/92853,"34615, 34616"
4,92856,13450,12885,1,Position,Mitch McConnell has a position (Minority Lead...,Minority Leader,Minority Leader,True,<NA>,<NA>,<NA>,NaN,None,2007-00-00,2024-11-00,2024-10-25T02:46:39Z,https://littlesis.org/relationships/92856,"13450, 12885"
5,117104,13762,14640,1,Position,Tom Daschle has a position (Secretary - desig...,Secretary - designate,Secretary - designate,True,<NA>,<NA>,<NA>,NaN,None,2008-11-00,2009-02-03,2009-02-04T09:17:23Z,https://littlesis.org/relationships/117104,"13762, 14640"
6,133310,3289,168,1,Position,David J Lesar has a position (COO) at Hallib...,COO,COO,True,True,True,True,NaN,None,1997-00-00,2000-00-00,2015-03-31T17:52:36Z,https://littlesis.org/relationships/133310,"3289, 168"
7,264030,58489,2,1,Position,Jack P Williams Jr has a position (Senior Vic...,Senior Vice President; Executive Officer,Executive Officer,True,False,True,<NA>,NaN,None,None,2015-00-00,2020-09-16T19:25:20Z,https://littlesis.org/relationships/264030,"58489, 2"
8,264035,58492,2,1,Position,James M Spellings Jr has a position (Vice Pre...,Vice President and General Tax Counsel; Vice P...,Vice President,True,False,True,<NA>,NaN,None,None,2015-00-00,2020-09-16T19:25:20Z,https://littlesis.org/relationships/264035,"58492, 2"
9,386325,15067,90705,1,Position,Richard Kinder has a position (Chairman & Chi...,Chairman & Chief Executive Officer; CEO,CEO,True,True,<NA>,<NA>,NaN,None,None,2015-00-00,2020-09-16T19:13:15Z,https://littlesis.org/relationships/386325,"15067, 90705"


In [29]:
# is current + is board 
q= "SELECT distinct e.* FROM entities e LEFT JOIN relationships r on e.ID = r.entity1_ID or e.ID = r.entity2_ID where category_id = 5 "
duckdb.query(q).to_df()



,id,name,aliases,type,types_other,description,updated,links,source
0,28670,Democratic Congressional Campaign Committee - ...,C00347864;DCCC;Democratic Congressional Campai...,Org,Organization; Political Fundraising Committee;...,None,2025-01-23T15:20:39Z,https://littlesis.org/entities/28670-Democrati...,Littlesis
1,29038,Searchlight Leadership Fund,Searchlight Leadership Fund,Org,Organization; Political Fundraising Committee;...,Affiliate: Harry Reid (D-Nev),2020-05-04T17:33:37Z,https://littlesis.org/entities/29038-Searchlig...,Littlesis
2,13271,John Ensign,John Ensign;John Eric Ensign,Person,Person; Political Candidate; Elected Represent...,former US Senator and Representative from Nevada,2024-11-21T17:49:54Z,https://littlesis.org/entities/13271-John_Ensign,Littlesis
3,13935,Al D’Amato,Al D’Amato;Alfonse D'Amato;Alfonse Damato;Alfo...,Person,Person; Political Candidate; Elected Represent...,US Senator from New York; Park Strategies,2021-03-17T20:28:22Z,https://littlesis.org/entities/13935-Al_D%E2%8...,Littlesis
4,13541,Harry Reid,Harry Reid;Harry Reid,Person,Person; Political Candidate; Elected Represent...,Late US Senator and Representative from Nevada,2025-01-23T20:47:52Z,https://littlesis.org/entities/13541-Harry_Reid,Littlesis
...,...,...,...,...,...,...,...,...,...
2176,451123,Cornelia Matson,Cornelia Long Matson;Cornelia Matson,Person,Person,Sarasota Florida,2024-10-29T22:01:43Z,https://littlesis.org/entities/451123-Cornelia...,Littlesis
2177,451583,Laura Ramsey,Laura Ramsey,Person,Person,"Retired, Shiloh Ohio",2024-11-21T18:07:40Z,https://littlesis.org/entities/451583-Laura_Ra...,Littlesis
2178,454577,John D Giles,John D Giles,Person,Person; Business Person,"Butler Business Systems - Butler, Pennsylvania",2025-01-07T16:56:50Z,https://littlesis.org/entities/454577-John_D_G...,Littlesis
2179,454798,Kevin Delbridge,Kevin Delbridge,Person,Person,Retired. Naples Florida,2025-01-20T20:02:35Z,https://littlesis.org/entities/454798-Kevin_De...,Littlesis


In [30]:
q= "SELECT * FROM relationships where category_id in (11,7)"
duckdb.query(q).to_df()

,id,entity1_id,entity2_id,category_id,category_desc,description,description_1,description_2,is_current,is_board,is_executive,is_employee,amount,currency,start_date,end_date,updated,link,id_all
0,872676,437,2,11,Hierarchy,XTO Energy Inc. and ExxonMobil have a hiera...,,None,True,<NA>,<NA>,<NA>,NaN,None,None,None,2014-12-19T00:20:47Z,https://littlesis.org/relationships/872676,"437, 2"
1,872677,87132,2,11,Hierarchy,ExxonMobil Canada Ltd. and ExxonMobil have ...,,None,True,<NA>,<NA>,<NA>,NaN,None,None,None,2014-12-19T00:20:47Z,https://littlesis.org/relationships/872677,"87132, 2"
2,872678,103066,2,11,Hierarchy,Mobil International and ExxonMobil have a h...,,None,True,<NA>,<NA>,<NA>,NaN,None,None,None,2014-12-19T00:20:47Z,https://littlesis.org/relationships/872678,"103066, 2"
3,872679,108125,2,11,Hierarchy,Creole Petroleum Corporation and ExxonMobil ...,,None,True,<NA>,<NA>,<NA>,NaN,None,None,None,2014-12-19T00:20:47Z,https://littlesis.org/relationships/872679,"108125, 2"
4,872680,111585,2,11,Hierarchy,"ExxonMobil Petroleum & Chemical, BVBA and Ex...",,None,True,<NA>,<NA>,<NA>,NaN,None,None,None,2014-12-19T00:20:47Z,https://littlesis.org/relationships/872680,"111585, 2"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,1999640,455251,143943,11,Hierarchy,Active Social Engineering Defense (ASED) and ...,,None,<NA>,<NA>,<NA>,<NA>,NaN,None,None,None,2025-02-24T03:27:00Z,https://littlesis.org/relationships/1999640,"455251, 143943"
217,1999654,455251,14711,11,Hierarchy,Active Social Engineering Defense (ASED) and ...,,None,<NA>,<NA>,<NA>,<NA>,NaN,None,None,None,2025-02-24T07:41:43Z,https://littlesis.org/relationships/1999654,"455251, 14711"
218,1999981,455281,455251,11,Hierarchy,PREVENTING INFORMATION REMOVAL AND NABBING HAR...,,None,<NA>,<NA>,<NA>,<NA>,NaN,None,None,None,2025-02-24T17:44:38Z,https://littlesis.org/relationships/1999981,"455281, 455251"
219,1999984,455282,455251,11,Hierarchy,Strategies for Investigating and Eliciting Inf...,,None,<NA>,<NA>,<NA>,<NA>,NaN,None,None,None,2025-02-24T17:57:11Z,https://littlesis.org/relationships/1999984,"455282, 455251"


In [31]:
duckdb.query("SELECT * FROM entities where lower(name) like 'wayne%'")
duckdb.query("SELECT * FROM relationships where id_all like '%138528%'")

┌─────────┬────────────┬────────────┬─────────────┬────────────────┬─────────────────────────────────────────────────────────────────────────────────────┬───────────────────────┬───────────────┬────────────┬──────────┬──────────────┬─────────────┬─────────┬──────────┬────────────┬────────────┬──────────────────────┬─────────────────────────────────────────────┬───────────────┐
│   id    │ entity1_id │ entity2_id │ category_id │ category_desc  │                                     description                                     │     description_1     │ description_2 │ is_current │ is_board │ is_executive │ is_employee │ amount  │ currency │ start_date │  end_date  │       updated        │                    link                     │    id_all     │
│  int64  │   int64    │   int64    │    int64    │    varchar     │                                       varchar                                       │        varchar        │    varchar    │  boolean   │ boolean  │   boolean    │   bool

# Lobbyview

## Import

In [32]:
lv_reports = pd.read_csv("reports.csv")
lv_clients = pd.read_csv("clients.csv")
lv_issue_text = pd.read_csv("issue_text.csv")
#lv_bills = pd.read_csv("bills.csv")
#lv_issues = pd.read_csv("issues.csv")

In [33]:
lv_reports.shape

(1690362, 11)

In [34]:
# odflitrovaní reportů pouze od roku 2016
lv_reports = lv_reports.loc[(lv_reports["filing_year"]) >=2016 ]

In [35]:
lv_clients

,lob_id,gvkey,bvdid,naics,client_name
0,000065ee-5384-579e-b167-c3f782cd0d55,NaN,NaN,NaN,KALYPSO
1,00031e5d-fb07-526f-be41-0b627892e042,NaN,NaN,NaN,PIGEONLY
2,00046722-0148-5d5d-b6a1-770fdf604147,NaN,NaN,NaN,CITY OF OCEAN CITY NEW JERSEY
3,0005ff7f-6811-532a-bca3-95557fba29a8,NaN,NaN,NaN,"SHELTERS INTERNATIONAL, LLC."
4,0007264e-791e-5727-97dd-4c88f2ad9807,NaN,NaN,NaN,POLICY INSTITUTE FOR RELIGION AND STATE
...,...,...,...,...,...
48522,fffaa78b-38d2-5aff-beb5-bbf6d2b6c7fc,24861.0,US383942097,713940.0,"PLANET FITNESS, INC."
48523,fffcf360-06b2-5a06-9eb3-396ff72ac2e4,NaN,NaN,NaN,AVI BERNSTEIN
48524,fffe34ed-3548-5caf-a4a8-84598b056728,NaN,NaN,NaN,TULSA AIRPORT AUTHORITY
48525,fffe422c-98c0-5bcb-a892-6784c3754a8a,NaN,NaN,NaN,360 QUALITY OF LIFE SOLUTIONS (A TRADE ASSOCIA...


In [36]:
lv_reports

,report_uuid,lob_id,registrant_id,registrant_name,filing_year,filing_period_code,amount,is_no_activity,is_self_filer,is_amendment,filing_url
5,15a9cb9e-a0fa-49fe-8c49-c548e9e72903,e9aecce5-0a61-50ad-a2fa-19777ff38542,157,"ADAMS AND REESE, LLP",2022,Q3,20000.0,False,NaN,False,https://lda.senate.gov/filings/public/filing/1...
8,4ebd73ee-83d4-404d-9efc-a948ff3c1ffb,e9aecce5-0a61-50ad-a2fa-19777ff38542,157,"ADAMS AND REESE, LLP",2016,Q1,32000.0,False,NaN,False,https://lda.senate.gov/filings/public/filing/4...
9,55a4f081-c02e-4312-bcfe-9b6fba434679,e9aecce5-0a61-50ad-a2fa-19777ff38542,157,"ADAMS AND REESE, LLP",2016,Q4,32000.0,False,NaN,False,https://lda.senate.gov/filings/public/filing/5...
10,577a9861-7dde-4e95-9f60-4439d4853c3d,e9aecce5-0a61-50ad-a2fa-19777ff38542,157,"ADAMS AND REESE, LLP",2016,Q2,32000.0,False,NaN,False,https://lda.senate.gov/filings/public/filing/5...
11,39758514-5b22-4e58-bc58-4c87840209ac,e9aecce5-0a61-50ad-a2fa-19777ff38542,157,"ADAMS AND REESE, LLP",2016,Q3,32000.0,False,NaN,False,https://lda.senate.gov/filings/public/filing/3...
...,...,...,...,...,...,...,...,...,...,...,...
1690303,dcf78bf7-f9b5-4845-a3d3-844e0a30ccf8,2e6bf10b-802f-56ec-983f-b417d350ad84,157,"ADAMS AND REESE, LLP",2021,Q1,10000.0,False,NaN,False,https://lda.senate.gov/filings/public/filing/d...
1690304,7658d509-9815-45e8-823e-a6e69f7ab5b9,2e6bf10b-802f-56ec-983f-b417d350ad84,157,"ADAMS AND REESE, LLP",2023,Q1,10000.0,False,NaN,False,https://lda.senate.gov/filings/public/filing/7...
1690305,4a146d53-ff80-49a1-bcfe-223495c4fc03,2e6bf10b-802f-56ec-983f-b417d350ad84,157,"ADAMS AND REESE, LLP",2023,Q2,10000.0,False,NaN,False,https://lda.senate.gov/filings/public/filing/4...
1690306,84a6e67d-4160-4c3a-b49f-3e0ac948756b,2e6bf10b-802f-56ec-983f-b417d350ad84,157,"ADAMS AND REESE, LLP",2023,Q3,10000.0,False,NaN,False,https://lda.senate.gov/filings/public/filing/8...


In [37]:
lv_issue_text

,report_uuid,general_issue_code,issue_ordinal_position,bill_id_agg,issue_text
0,000018d7-73cf-403c-bafb-e423afd81c67,MAN,0,NaN,Issues related to the CHIPS for America Act an...
1,000018d7-73cf-403c-bafb-e423afd81c67,SCI,0,NaN,Issues related to technological innovations in...
2,000028d3-9c4e-4a4e-9a76-c1d43456c024,FUE,0,NaN,Issues related to biofuels legislation and reg...
3,000028d3-9c4e-4a4e-9a76-c1d43456c024,ENG,0,NaN,Issues related to biofuels legislation and reg...
4,000028d3-9c4e-4a4e-9a76-c1d43456c024,DEF,0,NaN,Issues related to biofuels legislation and reg...
...,...,...,...,...,...
3161602,ffffe8ae-6894-4cb8-854f-9599aecd6dfb,TAX,1,NaN,"Tax reform legislation, generally."
3161603,ffffe8ae-6894-4cb8-854f-9599aecd6dfb,LBR,2,NaN,Issues related to antitrust
3161604,ffffe8ae-6894-4cb8-854f-9599aecd6dfb,RET,3,NaN,Legislation to enhance retirement plan options.
3161605,ffffe8ae-6894-4cb8-854f-9599aecd6dfb,FIN,4,{hjres66-115},Issues related to the regulation of asset mana...


## Matching

In [38]:
def clean_string(name):
    tokens = name.lower().replace(",", "").replace(".", "").replace("&", "and").split()
    return " ".join(t for t in tokens if t not in COMMON_WORDS)

def has_token_overlap(a, b, min_overlap=2):
    tokens_a = set(clean_string(a).split())
    tokens_b = set(clean_string(b).split())
    return len(tokens_a & tokens_b) >= min_overlap or len(tokens_a) == 1 #returns how many tokens the two names have in common, except when length 1

def safe_match(name, cleaned_dict, tsr_threshold=75, ratio_threshold = 0 , min_overlap=2):
    cleaned_input = clean_string(name)

    # Perform fuzzy matching against cleaned values
    result = process.extractOne(cleaned_input, cleaned_dict, scorer=fuzz.token_sort_ratio)

    if result:
        matched_cleaned_value , tsr_score, (match_id, match_name) = result
        ratio_score = fuzz.ratio(name.lower(), match_name.lower())

        if tsr_score >= tsr_threshold and has_token_overlap(name, match_name, min_overlap)  and ratio_score >= ratio_threshold:
            if isinstance(match_id,float): 
                print(match_id)
            return match_id, match_name, tsr_score, ratio_score

    return None, None, None, None

In [39]:
# common words to exclude/clean 
COMMON_WORDS = {
    "inc", "llc", "ltd", "company", "group", "partners",
    "corp", "corporation", "co", "the", "and", "&", "lp"
}

### Fuzzy matching by registrant

In [40]:
distinct_entity = duckdb.query("SELECT distinct id, name FROM entities where Type='Org' ").to_df()

In [41]:
distinct_registrant = duckdb.query("SELECT distinct registrant_id, registrant_name FROM lv_reports").to_df()
# dictionary 
lv_entity_cleaned = {
    (row["registrant_id"], row["registrant_name"]): clean_string(row["registrant_name"])
    for _, row in distinct_registrant.iterrows()
}

In [43]:
fuzzy_matching = distinct_entity
fuzzy_matching[["registrant_id","match","tsr_score","ratio_score"]] = fuzzy_matching["name"].apply(lambda x: pd.Series(safe_match(x,lv_entity_cleaned)) )

In [44]:
# omezit na vysledky s tsr_score >= 80% 
fuzzy_matching_filtered = duckdb.query("SELECT * FROM fuzzy_matching where match is not null and tsr_score >=80 order by tsr_score, ratio_score").to_df()

In [45]:
# convert registrant_id back to int64
null_count = int(fuzzy_matching_filtered["registrant_id"].isnull().sum())

if null_count == 0:
    fuzzy_matching_filtered["registrant_id"] = fuzzy_matching_filtered[["registrant_id"]].astype("Int64")
    print("Registrant_id OK")
else: raise ValueError("Registrant_id contains float")

Registrant_id OK


### Fuzzy matching by client

In [46]:
distinct_client = duckdb.query("SELECT  distinct lob_id, client_name FROM lv_clients ").to_df()

In [48]:
lv_client_cleaned = {
    (row["lob_id"], row["client_name"]): clean_string(row["client_name"])
    for _, row in distinct_client.iterrows()
}

fuzzy_matching_client = distinct_entity
fuzzy_matching_client[["lob_id","match","tsr_score","ratio_score"]] = fuzzy_matching_client["name"].apply(lambda x: pd.Series(safe_match(x,lv_client_cleaned,tsr_threshold=90)) )

In [49]:
lv_client_cleaned

{('00094d9a-15d6-540e-9c41-15f94e63e1a2',
  'SRI LANKA APPAREL EXPORTERS ASSN'): 'sri lanka apparel exporters assn',
 ('0009b134-7190-564a-b9fe-b81bf72c53c4',
  'AIR SECURITY INTL'): 'air security intl',
 ('00248509-dfce-55e1-8059-c886bb15de9f', 'INFOCYTE, INC.'): 'infocyte',
 ('002bb9f2-63df-597c-bc66-4648f8675670',
  'FORT BELKNAP INDIAN COMMUNITY AND ISLAND MOUNTAIN DEVELOPMENT GROUP'): 'fort belknap indian community island mountain development',
 ('0033ca9a-b43c-5114-8bea-a49dfe3b6cf7',
  'FABIANI & COMPANY ON BEHALF OF NATIONAL CHILDHOOD CANCER FOUNDATION'): 'fabiani on behalf of national childhood cancer foundation',
 ('003ff020-c890-52e5-90d9-749e40d3af0c',
  'GRAYS HARBOR PAPER'): 'grays harbor paper',
 ('0043cba3-4bcb-504d-8851-3605a2fc2b03', 'US CELLULAR'): 'us cellular',
 ('00556832-9de8-50f7-8bd1-3c7978e19338',
  'GULF STREAM DEVELOPMENT GROUP'): 'gulf stream development',
 ('00561c67-7970-56bf-a481-e3391f14f133', 'RADIATE GROUP'): 'radiate',
 ('0057ef85-4494-56ac-a94f-8175

## Exporting matching results

In [50]:
fuzzy_matching_filtered.to_csv("adv_matching_filtered.csv",index=False)
fuzzy_matching_client.to_csv("client_matching.csv",index=False)

## Import manual matching

In [51]:
matching_final = pd.read_excel("matching_final.xlsx")
matching_final = duckdb.query("SELECT * FROM matching_final where keep_flag = 'Y' or tsr_score = 100").to_df()
matching_final

,ID,Name,registrant_id,match,tsr_score,ratio_score,keep_flag
0,347,PPL,31086,PPL CORPORATION,100.000000,33.333333,None
1,145,Xerox,42001,XEROX CORPORATION,100.000000,45.454545,None
2,502,Scana,34466,SCANA CORPORATION,100.000000,45.454545,None
3,71,Sysco,401103671,SYSCO CORPORATION,100.000000,45.454545,None
4,459,Timken,38153,THE TIMKEN COMPANY,100.000000,50.000000,None
...,...,...,...,...,...,...,...
316,222539,Rose Law Firm,401106170,"THE ROSS LAW FIRM, LLC",92.307692,68.571429,Y
317,329067,"Vorys, Sater, Seymour and Pease",40236,"VORYS, SATER, SEYMOUR AND PEASE LLP",92.592593,93.939394,Y
318,26640,Cadwalader Wickersham & Taft,7645,"CADWALADER, WICKERSHAM & TAFT LLP",92.857143,91.803279,Y
319,41453,University of Texas Austin,40013107,THE UNIVERSITY OF TEXAS AT AUSTIN,94.545455,88.135593,Y


In [52]:
matching_final_clients = pd.read_csv("client_matching.csv")
matching_final_clients = duckdb.query("SELECT *FROM matching_final_clients where match is not null and tsr_score >95 order by tsr_score").to_df()
matching_final_clients 

,id,name,registrant_id,match,tsr_score,ratio_score,lob_id
0,2,ExxonMobil,14017.0,EXXON MOBIL CORPORATION,95.238095,60.606061,b9eeff59-e365-52d7-9cdf-b3a592d4e07a
1,68095,The Estée Lauder Companies Inc,NaN,"THE ESTEE LAUDER COMPANIES, INC",95.454545,95.081967,904930e4-5d85-5b7d-a5db-256de8c51f53
2,280284,ObjectVideo,NaN,OBJECT VIDEO,95.652174,95.652174,d7728dfd-efff-5fba-829b-7762fb2a076e
3,78575,"NBCUniversal, LLC",NaN,NBC UNIVERSAL,96.000000,80.000000,25d3c718-e134-5a0e-a4c8-2782acd95219
4,423225,United Against Nuclear Iran,NaN,UNITED AGAINST A NUCLEAR IRAN,96.428571,96.428571,cb54a625-cb27-5425-bada-af9ad60eb0c9
...,...,...,...,...,...,...,...
627,351432,Carbyne,NaN,CARBYNE INC.,100.000000,73.684211,f0288d93-e2fd-595a-b9d3-de8e78194176
628,368170,"Las Vegas Sands, Inc.",NaN,LAS VEGAS SANDS CORPORATION,100.000000,75.000000,5ba97d59-eb3c-5cdf-b958-34c108cbd8da
629,386927,Third Option Foundation,NaN,THIRD OPTION FOUNDATION,100.000000,100.000000,9edcff5e-c6e6-596f-bc8b-28122cd8788e
630,427139,Viridi Parente,401105993.0,"VIRIDI PARENTE, INC.",100.000000,82.352941,877ac008-05d3-5e55-9911-7b9d34cf022b


In [53]:
# SELECT reportu ktere maji pripojene littlesis ID (na registranta)
lobbying_reports_r = duckdb.query("SELECT mf.ID as littlesis_ID, re.* FROM lv_reports re inner join matching_final mf on re.registrant_id = mf.registrant_id where mf.ID is not null").to_df()
# pripojeni client_name z lv_clients (pres lob_id)
lobbying_reports_r = duckdb.query("SELECT l.*, c.client_name FROM lobbying_reports_r l left join lv_clients c on l.lob_id = c.lob_id ").to_df()


In [54]:
# SELECT reportu ktere maji pripojene littlesis ID (na clienta)
lobbying_reports_c = duckdb.query("SELECT mc.ID as littlesis_ID, re.* FROM lv_reports re left join matching_final_clients mc on re.lob_id = mc.lob_id where mc.ID is not null").to_df()
# pripojeni client_name z lv_clients (pres lob_id)
lobbying_reports_c = duckdb.query("SELECT l.*, c.client_name FROM lobbying_reports_c l left join lv_clients c on l.lob_id = c.lob_id ").to_df()

In [55]:
lobbying_reports_c

,littlesis_ID,report_uuid,lob_id,registrant_id,registrant_name,filing_year,filing_period_code,amount,is_no_activity,is_self_filer,is_amendment,filing_url,client_name
0,88,aabc5e5c-3d8e-4715-b4c3-40ffc15ab023,612ef4ed-22c1-5454-93a4-005b867899a4,401107502,"MORAN GLOBAL STRATEGIES, INC.",2023,Q2,NaN,False,False,False,https://lda.senate.gov/filings/public/filing/a...,GENERAL DYNAMICS
1,61,a23c1444-6c53-49ae-9fb4-ca2360e14851,3e54f727-549c-5ab3-9073-522a811520e7,401104716,KLEIN/JOHNSON GROUP,2021,Q1,30000.0,False,<NA>,False,https://lda.senate.gov/filings/public/filing/a...,INTEL CORPORATION
2,61,50e14ae3-5eb1-48e4-8e01-efa993ea110d,3e54f727-549c-5ab3-9073-522a811520e7,401104716,KLEIN/JOHNSON GROUP,2021,Q4,50000.0,False,<NA>,False,https://lda.senate.gov/filings/public/filing/5...,INTEL CORPORATION
3,61,11e001e5-7910-4be0-bce6-55e64f53853f,3e54f727-549c-5ab3-9073-522a811520e7,401104716,KLEIN/JOHNSON GROUP,2021,Q2,30000.0,False,<NA>,False,https://lda.senate.gov/filings/public/filing/1...,INTEL CORPORATION
4,61,a6256a5c-f81a-408c-88fe-463bb683e0d4,3e54f727-549c-5ab3-9073-522a811520e7,401104716,KLEIN/JOHNSON GROUP,2021,Q3,50000.0,False,<NA>,False,https://lda.senate.gov/filings/public/filing/a...,INTEL CORPORATION
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47065,217668,a82d39e7-139e-4f32-87af-9184736a1c81,cb4ac22a-dd82-5a42-a289-83fd13ba07d0,401104556,BOUNDARY STONE PARTNERS,2022,Q2,60000.0,False,False,False,https://lda.senate.gov/filings/public/filing/a...,"STRIPE, INC."
47066,217668,6780c687-9e4a-4cc2-8869-45c5fc1af5bb,cb4ac22a-dd82-5a42-a289-83fd13ba07d0,401104556,BOUNDARY STONE PARTNERS,2022,Q3,60000.0,False,False,False,https://lda.senate.gov/filings/public/filing/6...,"STRIPE, INC."
47067,217668,908bb8f5-ebd7-4e60-a839-fdf62c722367,cb4ac22a-dd82-5a42-a289-83fd13ba07d0,401104556,BOUNDARY STONE PARTNERS,2023,Q1,NaN,True,False,False,https://lda.senate.gov/filings/public/filing/9...,"STRIPE, INC."
47068,217668,2af4c71a-9140-44a1-a8e5-00e8f0013eb7,cb4ac22a-dd82-5a42-a289-83fd13ba07d0,401104556,BOUNDARY STONE PARTNERS,2022,Q1,NaN,False,False,True,https://lda.senate.gov/filings/public/filing/2...,"STRIPE, INC."


In [56]:
#odmazání duplicit
lobbying_reports_c = duckdb.query("SELECT *FROM lobbying_reports_c where registrant_name != client_name ").to_df()
lobbying_reports_c["is_self_filer"] = False

In [57]:
def self_filer_fix(lobbying_reports):
    lobbying_reports["is_self_filer"] =lobbying_reports["is_self_filer"].fillna(False)

    #if client_name empty -> registrant_name
    lobbying_reports["client_name"] = lobbying_reports.apply(
        lambda row: row["registrant_name"] if pd.isna(row["client_name"]) else row["client_name"],
        axis=1
    )


    # if is_self_filer false or empty and registrant = client name set is_self_filer as True
    lobbying_reports["is_self_filer"] = lobbying_reports.apply(
        lambda row: True if (
            row["registrant_name"] == row["client_name"] and 
            (row["is_self_filer"] == False or pd.isna(row["is_self_filer"]))
        ) else row["is_self_filer"],
        axis=1
    )
    
    return lobbying_reports


In [58]:
lobbying_reports_r = self_filer_fix(lobbying_reports_r)
lobbying_reports_c = self_filer_fix(lobbying_reports_c)

In [59]:
# dataset s popisy problematky jednotlivých reportů
issue_codebook = pd.read_excel("issue_cdbk.xlsx")

merged = duckdb.query("SELECT * FROM lobbying_reports_r union all SELECT * FROM  lobbying_reports_c").to_df()

#pripojeni issue text k reportům + ciselnik k GIC
report_issues = duckdb.query("""
             SELECT i.report_uuid, i.general_issue_code, cd.description as description, i.bill_id_agg, i.issue_text
             FROM lv_issue_text i 
             left join issue_codebook cd on i.general_issue_code = cd.code
             inner join merged m on m.report_uuid = i.report_uuid
             """ ).to_df()

report_issues["bill_id_agg"]= report_issues["bill_id_agg"].str.replace(r'[{}]', '', regex=True) # odstranění {}


In [60]:
report_issues

,report_uuid,general_issue_code,description,bill_id_agg,issue_text
0,39fd84b3-5b25-4ba5-b5b6-dcc9d737470d,SMB,Small Business,None,E-Commerce
1,39fed6ab-97bf-408a-bbd3-fabd607bbe71,COM,Communications/Broadcasting/Radio/TV,None,S. 1260 - United States Innovation and Competi...
2,39fed6ab-97bf-408a-bbd3-fabd607bbe71,CPI,Computer Industry,None,Government innovation; Innovation in the techn...
3,39fed6ab-97bf-408a-bbd3-fabd607bbe71,CPT,Copyright/Patent/Trademark,None,Intellectual Property issues.
4,39fed6ab-97bf-408a-bbd3-fabd607bbe71,DEF,Defense,None,Issues related to government contracting; Nati...
...,...,...,...,...,...
234872,fff717c5-10d6-40e3-a8c9-92d45123d9ab,TAX,Taxation/Internal Revenue Code,None,Corporate tax reform.
234873,fff717c5-10d6-40e3-a8c9-92d45123d9ab,EDU,Education,None,Computer science education.
234874,fff717c5-10d6-40e3-a8c9-92d45123d9ab,CPT,Copyright/Patent/Trademark,None,Patent legislation.
234875,fff717c5-10d6-40e3-a8c9-92d45123d9ab,IMM,Immigration,None,Immigration--high-skilled visas.


## Export final

In [61]:
#export
lobbying_reports_r.to_csv("neo_lobbying_reports.csv", index=False)
lobbying_reports_c.to_csv("neo_lobbying_clients.csv",index=False)
report_issues.to_csv("neo_report_issues.csv", index=False)

## Testing

In [64]:
duckdb.query("SELECT * FROM lobbying_reports_r where is_self_filer = 'False' ").to_df()

,littlesis_ID,report_uuid,lob_id,registrant_id,registrant_name,filing_year,filing_period_code,amount,is_no_activity,is_self_filer,is_amendment,filing_url,client_name
0,326262,c4fcfc56-f96d-4690-82ca-c4b744d73c71,5aac59cf-27c1-5adb-b7df-dfc96b60123c,401105189,STONINGTON GLOBAL,2021,Q1,45000.0,True,False,True,https://lda.senate.gov/filings/public/filing/c...,B & H FOTO & ELECTRONICS CORP.
1,326262,4777ae2d-f4f0-4117-bb46-7d783edfc628,5aac59cf-27c1-5adb-b7df-dfc96b60123c,401105189,STONINGTON GLOBAL,2021,Q1,45000.0,True,False,True,https://lda.senate.gov/filings/public/filing/4...,B & H FOTO & ELECTRONICS CORP.
2,326262,47334776-1188-4991-b854-406d9c1140de,5aac59cf-27c1-5adb-b7df-dfc96b60123c,401105189,STONINGTON GLOBAL,2021,Q1,45000.0,True,False,False,https://lda.senate.gov/filings/public/filing/4...,B & H FOTO & ELECTRONICS CORP.
3,326262,b3eaf19d-fe3e-456e-9f9d-e7ab38b95b22,5aac59cf-27c1-5adb-b7df-dfc96b60123c,401105189,STONINGTON GLOBAL,2021,Q4,45000.0,True,False,False,https://lda.senate.gov/filings/public/filing/b...,B & H FOTO & ELECTRONICS CORP.
4,326262,5a1dd3f9-c987-4f14-a64f-7f770b908a18,5aac59cf-27c1-5adb-b7df-dfc96b60123c,401105189,STONINGTON GLOBAL,2021,Q2,45000.0,True,False,False,https://lda.senate.gov/filings/public/filing/5...,B & H FOTO & ELECTRONICS CORP.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66612,21559,ed9680fc-e81e-4aee-8b72-9348168bf891,292d333f-13df-5d2a-a231-7dfa64154c66,2194,AMERICAN DEFENSE INTERNATIONAL,2020,Q2,NaN,True,False,False,https://lda.senate.gov/filings/public/filing/e...,SENSIS CORPORATION
66613,15859,f37d09cd-691c-4d6c-b000-3b65927bce65,b433a883-f9f5-5f06-8407-7fc941cc8dac,11195,COVINGTON & BURLING LLP,2020,Q1,NaN,True,False,False,https://lda.senate.gov/filings/public/filing/f...,ASSOCIATION OF EQUIPMENT MANUFACTURERS
66614,23624,a2f3e4c9-7afe-450f-8475-e0290a261999,d1d7627c-30db-5362-8c55-8be0b2b4d327,15042,FOLEY & LARDNER LLP,2023,Q2,NaN,True,False,False,https://lda.senate.gov/filings/public/filing/a...,"DEPHY, INC."
66615,23624,b4d071ac-e355-461b-9a79-8bbb2e7155af,d1d7627c-30db-5362-8c55-8be0b2b4d327,15042,FOLEY & LARDNER LLP,2023,Q3,NaN,True,False,False,https://lda.senate.gov/filings/public/filing/b...,"DEPHY, INC."


In [65]:
duckdb.query("""
             select count(distinct report_uuid),count(distinct registrant_id),count(distinct lob_id), 'r' as src from lobbying_reports_r
   
             """)

┌─────────────────────────────┬───────────────────────────────┬────────────────────────┬─────────┐
│ count(DISTINCT report_uuid) │ count(DISTINCT registrant_id) │ count(DISTINCT lob_id) │   src   │
│            int64            │             int64             │         int64          │ varchar │
├─────────────────────────────┼───────────────────────────────┼────────────────────────┼─────────┤
│                       74181 │                           313 │                   4835 │ r       │
└─────────────────────────────┴───────────────────────────────┴────────────────────────┴─────────┘

In [66]:
duckdb.query(""" 
             select count(distinct report_uuid),count(distinct registrant_id),count(distinct lob_id), 'c' as src from lobbying_reports_c
""")

┌─────────────────────────────┬───────────────────────────────┬────────────────────────┬─────────┐
│ count(DISTINCT report_uuid) │ count(DISTINCT registrant_id) │ count(DISTINCT lob_id) │   src   │
│            int64            │             int64             │         int64          │ varchar │
├─────────────────────────────┼───────────────────────────────┼────────────────────────┼─────────┤
│                       41939 │                           916 │                    465 │ c       │
└─────────────────────────────┴───────────────────────────────┴────────────────────────┴─────────┘

In [68]:
merged

,littlesis_ID,report_uuid,lob_id,registrant_id,registrant_name,filing_year,filing_period_code,amount,is_no_activity,is_self_filer,is_amendment,filing_url,client_name
0,326262,c4fcfc56-f96d-4690-82ca-c4b744d73c71,5aac59cf-27c1-5adb-b7df-dfc96b60123c,401105189,STONINGTON GLOBAL,2021,Q1,45000.0,True,False,True,https://lda.senate.gov/filings/public/filing/c...,B & H FOTO & ELECTRONICS CORP.
1,326262,4777ae2d-f4f0-4117-bb46-7d783edfc628,5aac59cf-27c1-5adb-b7df-dfc96b60123c,401105189,STONINGTON GLOBAL,2021,Q1,45000.0,True,False,True,https://lda.senate.gov/filings/public/filing/4...,B & H FOTO & ELECTRONICS CORP.
2,326262,47334776-1188-4991-b854-406d9c1140de,5aac59cf-27c1-5adb-b7df-dfc96b60123c,401105189,STONINGTON GLOBAL,2021,Q1,45000.0,True,False,False,https://lda.senate.gov/filings/public/filing/4...,B & H FOTO & ELECTRONICS CORP.
3,326262,b3eaf19d-fe3e-456e-9f9d-e7ab38b95b22,5aac59cf-27c1-5adb-b7df-dfc96b60123c,401105189,STONINGTON GLOBAL,2021,Q4,45000.0,True,False,False,https://lda.senate.gov/filings/public/filing/b...,B & H FOTO & ELECTRONICS CORP.
4,326262,5a1dd3f9-c987-4f14-a64f-7f770b908a18,5aac59cf-27c1-5adb-b7df-dfc96b60123c,401105189,STONINGTON GLOBAL,2021,Q2,45000.0,True,False,False,https://lda.senate.gov/filings/public/filing/5...,B & H FOTO & ELECTRONICS CORP.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117283,217668,a82d39e7-139e-4f32-87af-9184736a1c81,cb4ac22a-dd82-5a42-a289-83fd13ba07d0,401104556,BOUNDARY STONE PARTNERS,2022,Q2,60000.0,False,False,False,https://lda.senate.gov/filings/public/filing/a...,"STRIPE, INC."
117284,217668,6780c687-9e4a-4cc2-8869-45c5fc1af5bb,cb4ac22a-dd82-5a42-a289-83fd13ba07d0,401104556,BOUNDARY STONE PARTNERS,2022,Q3,60000.0,False,False,False,https://lda.senate.gov/filings/public/filing/6...,"STRIPE, INC."
117285,217668,908bb8f5-ebd7-4e60-a839-fdf62c722367,cb4ac22a-dd82-5a42-a289-83fd13ba07d0,401104556,BOUNDARY STONE PARTNERS,2023,Q1,NaN,True,False,False,https://lda.senate.gov/filings/public/filing/9...,"STRIPE, INC."
117286,217668,2af4c71a-9140-44a1-a8e5-00e8f0013eb7,cb4ac22a-dd82-5a42-a289-83fd13ba07d0,401104556,BOUNDARY STONE PARTNERS,2022,Q1,NaN,False,False,True,https://lda.senate.gov/filings/public/filing/2...,"STRIPE, INC."


In [69]:
report_issues

,report_uuid,general_issue_code,description,bill_id_agg,issue_text
0,39fd84b3-5b25-4ba5-b5b6-dcc9d737470d,SMB,Small Business,None,E-Commerce
1,39fed6ab-97bf-408a-bbd3-fabd607bbe71,COM,Communications/Broadcasting/Radio/TV,None,S. 1260 - United States Innovation and Competi...
2,39fed6ab-97bf-408a-bbd3-fabd607bbe71,CPI,Computer Industry,None,Government innovation; Innovation in the techn...
3,39fed6ab-97bf-408a-bbd3-fabd607bbe71,CPT,Copyright/Patent/Trademark,None,Intellectual Property issues.
4,39fed6ab-97bf-408a-bbd3-fabd607bbe71,DEF,Defense,None,Issues related to government contracting; Nati...
...,...,...,...,...,...
234872,fff717c5-10d6-40e3-a8c9-92d45123d9ab,TAX,Taxation/Internal Revenue Code,None,Corporate tax reform.
234873,fff717c5-10d6-40e3-a8c9-92d45123d9ab,EDU,Education,None,Computer science education.
234874,fff717c5-10d6-40e3-a8c9-92d45123d9ab,CPT,Copyright/Patent/Trademark,None,Patent legislation.
234875,fff717c5-10d6-40e3-a8c9-92d45123d9ab,IMM,Immigration,None,Immigration--high-skilled visas.


In [70]:
report_issues.columns

Index(['report_uuid', 'general_issue_code', 'description', 'bill_id_agg',
       'issue_text'],
      dtype='object')